In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
import itertools
import pickle
import logging
from logging.handlers import RotatingFileHandler

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [3]:
random_seeds = np.random.randint(2343, 3485327, size=5)
random_seeds

array([3355825,  229561,  948006,  329412, 1453922])

In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('lof_nested_cv.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [5]:
def nested_cv(X, random_seeds, n_outer_folds=5, n_inner_folds=3, pca_components=[], mod_selection_score=accuracy_score, positive_class=0):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    
    best_params = {'n_neighbors': '', 'contamination': 0, 'metric': 0, 'pca': 0, 'scaler': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'n_neighbors': '', 'contamination': 0, 'metric': 0, 'pca': 0, 'scaler': ''}

    y = X['Mezzo'].values
    y = [0. if x==positive_class else 1. for x in y]
    y = np.array(y, dtype=float)
    
    X = X.drop(columns='Mezzo').values

    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=n_outer_folds, shuffle=True, random_state=seed)
        
        for outer_cv_number, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=n_inner_folds, shuffle=True, random_state=seed)

            best_score = 0
            best_pipeline = Pipeline([('svm', LocalOutlierFactor())])
            for inner_cv_number, (trainval_idx, valid_idx) in enumerate(inner_cv.split(X_train, y_train)):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                
                for params in itertools.product(n_neighborss, contaminations, metrics, pca_components, scalers):
                    pipeline = Pipeline([
                        ('scaler', params[4]),
                        ('pca', PCA(n_components=params[3])),
                        ('lof', LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)) 
                    ])
                    
                    pipeline.fit(X_trainval)
                    
                    pred_values = pipeline.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]
                    
                    score = mod_selection_score(true_values, pred_values)
                    curr_params = {
                            'n_neighbors': params[0],
                            'contamination': params[1],
                            'metric': params[2],
                            'pca': params[3],
                            'scaler': params[4]
                        }
                    logging.info(f"inner cv number: {inner_cv_number}, {mod_selection_score.__name__}: {score}, with params: {curr_params}")
                        
                    if score > best_score:
                        best_score = score
                        best_pipeline = pipeline
                        best_params = curr_params

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            best_pipeline.fit(X_train)
            pred_values = best_pipeline.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy = accuracy_score(true_values, pred_values)
            precision = precision_score(true_values, pred_values, zero_division=0.0)
            recall = recall_score(true_values, pred_values)
            f1 = f1_score(true_values, pred_values)

            if accuracy > best_overall_accuracy:
                best_overall_accuracy = accuracy_score(true_values, pred_values)
                best_overall_params = best_params

            logging.info(f"outer cv number: {outer_cv_number}, accuracy: {score}, precision: {precision}, recall: {recall}, f1: {f1} with params: {curr_params}")

            accuracy_scores.append(accuracy)
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_overall_params['n_neighbors'],
            'best contamination': best_overall_params['contamination'],
            'best metric': best_overall_params['metric'],
            'best pca components': best_overall_params['pca'],
            'best scaler': best_overall_params['scaler'],
            'score used for model selection': mod_selection_score.__name__,
            'method used for model selection': 'nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'f1 mean': np.mean(f1_scores) * 100,
            'f1 std': np.std(f1_scores) * 100,
            'best overall accuracy': best_overall_accuracy * 100,
            'class': positive_class}

In [6]:
results = nested_cv(X, random_seeds, pca_components=[20,25,30,35], mod_selection_score=accuracy_score, positive_class=0)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_neighbors,best contamination,best metric,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,LocalOutlierFactor,30,0.45,cosine,20,StandardScaler(),accuracy_score,nested cv,64.0,10.817474,69.150039,10.289203,63.142857,19.449568,64.146806,13.329311,88.461538,0


In [7]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [8]:
scores_df = add_record(scores_df, nested_cv(X, random_seeds, pca_components=[20,25,30,35], mod_selection_score=f1_score, positive_class=0))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,LocalOutlierFactor,30,0.45,cosine,20,StandardScaler(),accuracy_score,nested cv,64.0,10.817474,69.150039,10.289203,63.142857,19.449568,64.146806,13.329311,88.461538,0
1,LocalOutlierFactor,30,0.45,cosine,20,StandardScaler(),f1_score,nested cv,62.0,11.164186,64.301181,10.236185,70.571429,18.661321,65.879491,11.530746,88.461538,0


In [9]:
file_path = 'lof_exp5_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)